In [1]:
#import libraries
import cvxpy as cp
import numpy as np
import pandas as pd
import cmath

#load data
xls = pd.ExcelFile('data\case6.xlsx')
branch = pd.read_excel(xls, 'branch')
gen = pd.read_excel(xls, 'gen')
bus = pd.read_excel(xls, 'bus')
cost = pd.read_excel(xls, 'cost')

#Y matrix
base = 100
bus_len = bus.shape[0]
gen_len = len(gen['bus'].ravel())

Ybus = np.matrix(np.zeros([bus_len,bus_len]),dtype=complex)
shunt = np.zeros(bus_len, dtype=complex)
x = [pd.DataFrame() for i in range(bus_len)]
for i in range(bus_len):  
    x[i] = branch.loc[branch['fbus'] == i]
    fro = [j for j in x[i]['tbus']]
    fro = pd.DataFrame(fro)
    for j in range(len(fro)):
        Ybus[i-1,fro.loc[j][0]-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        Ybus[fro.loc[j][0]-1,i-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])

for i in range(bus_len):
    Ybus[i,i] = np.sum(Ybus[i,:])*(-1) + complex(0,sum(branch.loc[(branch['fbus']==i+1) | (branch['tbus']==i+1)]['b'])/2)

In [129]:
# declare variables
Pd = np.zeros(bus_len)
Qd = np.zeros(bus_len)
Vmax = np.zeros(bus_len)
Vmin = np.zeros(bus_len)
Pmax = np.zeros(bus_len)
Pmin = np.zeros(bus_len)
Qmax = np.zeros(bus_len)
Qmin = np.zeros(bus_len)
cost_cons = np.zeros(gen_len)
cost_lin = np.zeros(gen_len)
cost_quad = np.zeros(gen_len)

#load data into variables
for i in range(bus_len):
    Pd[i] = bus['Pd'][i]/base
    Qd[i] = bus['Qd'][i]/base
    Vmax[i] = bus['Vmax'][i]
    Vmin[i] = bus['Vmin'][i]

for i in range(gen_len):
    k = i
    i = gen['bus'].ravel()[i]
    Pmax[i-1] = gen.loc[gen['bus'] == i]['Pmax'].ravel()[0]/base
    Pmin[i-1] = gen.loc[gen['bus'] == i]['Pmin'].ravel()[0]/base
    Qmax[i-1] = gen.loc[gen['bus'] == i]['Qmax'].ravel()[0]/base
    Qmin[i-1] = gen.loc[gen['bus'] == i]['Qmin'].ravel()[0]/base
    
    
for i in range(gen_len):
    cost_cons[i] = cost['cons'][i]
    cost_lin[i] = cost['lin'][i]*base
    cost_quad[i] = cost['quad'][i]*base**2 

In [130]:
#Load relaxation parameters
W = cp.Variable((2*bus_len, 2*bus_len), PSD=True)

e = ['' for i in range(bus_len)]
ee= ['' for i in range(bus_len)]
for i in range(bus_len):
    e[i] = np.matrix(np.zeros(bus_len))
    e[i][0,i] = 1
    ee[i] = np.transpose(e[i])*e[i]


l = [['' for i in range(len(bus))] for i in range(len(bus))]
L = [['' for i in range(len(bus))] for i in range(len(bus))]
L_ = [['' for i in range(len(bus))] for i in range(len(bus))]
M = ['' for i in range(len(bus))]
N = ['' for i in range(len(bus))]
p = ['' for i in range(len(bus))]
q = ['' for i in range(len(bus))]
for i in range(len(bus)):
    M[i] = np.concatenate([np.concatenate([ee[i],np.matrix(np.zeros([bus_len,bus_len]))],axis=1),
               np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),ee[i]],axis=1)])
    N[i] = np.concatenate([np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),np.matrix(np.zeros([bus_len,bus_len]))],axis=1),
               np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),ee[i]],axis=1)])
    p[i] = 0.5*np.concatenate([np.concatenate([np.real(np.transpose(ee[i]*Ybus) + ee[i]*Ybus),np.imag(np.transpose(ee[i]*Ybus) - ee[i]*Ybus)],axis=1),
                             np.concatenate([np.imag(ee[i]*Ybus - np.transpose(ee[i]*Ybus)), np.real(np.transpose(ee[i]*Ybus) + ee[i]*Ybus)],axis=1)])
    q[i] = -0.5*np.concatenate([np.concatenate([np.imag(np.transpose(ee[i]*Ybus) + ee[i]*Ybus),np.real(ee[i]*Ybus - np.transpose(ee[i]*Ybus))],axis=1),
                             np.concatenate([np.real(np.transpose(ee[i]*Ybus) - ee[i]*Ybus), np.imag(np.transpose(ee[i]*Ybus) + ee[i]*Ybus)],axis=1)])

for i in range(len(branch)):
    f = int(branch.iloc[i]['fbus'])
    t = int(branch.iloc[i]['tbus'])
    r = branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['r'][i]
    x = branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['x'][i]
    l[f-1][t-1] = (complex(0,branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['b'][i])/2 + 1/(complex(r,x)))*np.transpose(e[f-1])*e[f-1] - 1/(complex(r,x))*np.transpose(e[f-1])*e[t-1]
    l[t-1][f-1] = (complex(0,branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['b'][i])/2 + 1/(complex(r,x)))*np.transpose(e[t-1])*e[t-1] - 1/(complex(r,x))*np.transpose(e[t-1])*e[f-1]
    L_[f-1][t-1] = -0.5*np.concatenate([np.concatenate([np.imag(l[f-1][t-1] + np.transpose(l[f-1][t-1])),np.real(-np.transpose(l[f-1][t-1]) + l[f-1][t-1])],axis=1),
                             np.concatenate([np.real(-l[f-1][t-1] + np.transpose(l[f-1][t-1])), np.imag(l[f-1][t-1] + np.transpose(l[f-1][t-1]))],axis=1)])
    L_[t-1][f-1] = -0.5*np.concatenate([np.concatenate([np.imag(l[t-1][f-1] + np.transpose(l[t-1][f-1])),np.real(-np.transpose(l[t-1][f-1]) + l[t-1][f-1])],axis=1),
                             np.concatenate([np.real(-l[t-1][f-1] + np.transpose(l[t-1][f-1])), np.imag(l[t-1][f-1] + np.transpose(l[t-1][f-1]))],axis=1)])
    L[f-1][t-1] = 0.5*np.concatenate([np.concatenate([np.real(l[f-1][t-1] + np.transpose(l[f-1][t-1])),np.imag(np.transpose(l[f-1][t-1]) - l[f-1][t-1])],axis=1),
                             np.concatenate([np.imag(l[f-1][t-1] - np.transpose(l[f-1][t-1])), np.real(l[f-1][t-1] + np.transpose(l[f-1][t-1]))],axis=1)])
    L[t-1][f-1] = 0.5*np.concatenate([np.concatenate([np.real(l[t-1][f-1] + np.transpose(l[t-1][f-1])),np.imag(np.transpose(l[t-1][f-1]) - l[t-1][f-1])],axis=1),
                             np.concatenate([np.imag(l[t-1][f-1] - np.transpose(l[t-1][f-1])), np.real(l[t-1][f-1] + np.transpose(l[t-1][f-1]))],axis=1)])

#Constraints
constraints = [(W >> 0)]
constraints += [W[:,bus_len] == 0]
constraints += [W[bus_len,:] == 0]
   
for i in range(bus_len):
    constraints += [cp.trace(M[i]@W) <= Vmax[i]**2]
    constraints += [cp.trace(M[i]@W) >= Vmin[i]**2]
    constraints += [cp.trace(p[i]@W) <= Pmax[i]-Pd[i]]
    constraints += [cp.trace(p[i]@W) >= Pmin[i]-Pd[i]]
    constraints += [cp.trace(q[i]@W) <= Qmax[i]-Qd[i]]
    constraints += [cp.trace(q[i]@W) >= Qmin[i]-Qd[i]]
 
#Flow constraints   
for i in range(len(branch)):
    f = int(branch.iloc[i]['fbus'])
    t = int(branch.iloc[i]['tbus']) 
    val = (branch.loc[i]['rateA']/base)
    if(val == 0):
        val = 100
    constraints += [cp.trace(L[f-1][t-1]@W)**2 + cp.trace(L_[f-1][t-1]@W)**2 <= (val)**2]
    constraints += [cp.trace(L[t-1][f-1]@W)**2 + cp.trace(L_[t-1][f-1]@W)**2 <= (val)**2]

In [165]:
points = 1000
load = ["" for i in range(bus_len)]
means = ["" for i in range(bus_len)]
std = ["" for i in range(bus_len)]
covs = np.zeros([bus_len,bus_len])
corr = 0.05

for i in range(bus_len):
    load[i] = np.random.uniform(0,1.5,points)
    means[i] = load[i].mean()
    std[i] = load[i].std()/bus_len
    
for i in range(bus_len):    
    for j in range(bus_len):
        if(i==j):
            covs[i,j] = std[i]*std[j]
        else:
            covs[i,j] = std[i]*std[j]*corr

profile = np.random.multivariate_normal(means, covs, 100)

In [185]:
counter  = 0 # Valid counter

Qd = np.array([0.  , 0.  , 0.  , 0.15, 0.15  , 0.15])
prop = ["" for j in range(len(profile))]
for i in range(len(profile)):
    Pd = profile[i]

    Pg = [0 for j in range(gen_len)]
    Qg = [0 for j in range(gen_len)]
    for j in range(gen_len):
        Pg[j] = cp.trace(p[gen['bus'][j]-1]@W) + Pd[gen['bus'][j]-1]
        Qg[j] = cp.trace(q[gen['bus'][j]-1]@W) + Qd[gen['bus'][j]-1]

    # Cost function
    objective = cp.Minimize(sum(cost_cons) + 
        (sum(Pg[j]*cost_lin[j] for j in range(gen_len))) + 
        (sum(cost_quad[j]*(Pg[j]**2) for j in range(gen_len)))
                           )

    #Objective function
    prob = cp.Problem(objective, constraints)
    # solve
    prob.solve(solver=cp.MOSEK)#, verbose=True)
    
    #Extract voltage phasors from W matrix
    [evl,evc] = np.linalg.eigh(W.value)
    eigWratio = evl[-1]/evl[-2]
    Vopt = np.sqrt(evl[-1])*evc[:,-1]
    Vopt = Vopt[0:bus_len] + [complex(0,Vopt[j]) for j in range(bus_len,2*bus_len)]
    abs(np.around(Vopt,decimals=4))    
    
    #Store values from convex relaxation
    vv = np.zeros(bus_len)
    qv = np.zeros(bus_len)
    pv = np.zeros(bus_len)
    tt = np.zeros(bus_len)
    for k in range(gen_len):
        pv[gen['bus'][k]-1] = Pg[gen['bus'][k]-1].value
        qv[gen['bus'][k]-1] = Qg[gen['bus'][k]-1].value
    for n in range(bus_len):
        tt[n] = cmath.polar(Vopt[n])[1]
        vv[n] = abs(np.around(Vopt,decimals=4))[n]

    p1 = ["" for k in range(len(branch))]
    p2 = ["" for k in range(bus_len)]
    p3 = ["" for k in range(bus_len)]
    
    #Check Line flow constraints
    for k in range(len(branch)): 
        x = int(branch.loc[k]['fbus'])-1
        y = int(branch.loc[k]['tbus'])-1
        p1[k] = abs((((vv[x])**2)*cmath.polar(Ybus[x,y])[0])*np.cos(cmath.polar(Ybus[x,y])[1]) - (vv[x])*(vv[y])*cmath.polar(Ybus[x,y])[0]*np.cos((tt[x])-(tt[y])-cmath.polar(Ybus[x,y])[1]))<= (branch.loc[k]['rateA'])/base

    #Check active/reactive power balance
    for k in range(bus_len):
        p2[k] = sum([(vv[k])*(vv[j])*cmath.polar(Ybus[k,j])[0]*np.cos((tt[k])-(tt[j])-cmath.polar(Ybus[k,j])[1]) for j in range(bus_len)]) - pv[k] + Pd[k] <=0.001
        p3[k] = sum([(vv[k])*(vv[j])*cmath.polar(Ybus[k,j])[0]*np.sin((tt[k])-(tt[j])-cmath.polar(Ybus[k,j])[1]) for j in range(bus_len)]) - qv[k] + Qd[k] <=0.001

    prop1 = np.all(p1)
    prop2 = np.all(p2)
    prop3 = np.all(p3)
    
    prop[i] = prop1 & prop2 & prop3
    if(prop[i]):
        counter = counter + 1

In [164]:
counter

754

In [187]:
counter

74

In [67]:
from pyDOE import *
lhs(10, samples=10)

array([[0.42274927, 0.30917752, 0.94165237, 0.95680044, 0.70894469,
        0.64807619, 0.62721945, 0.60587712, 0.12602805, 0.79981946],
       [0.55582285, 0.19024941, 0.21689915, 0.16741102, 0.65490955,
        0.27739424, 0.70346244, 0.77288296, 0.51842412, 0.46383748],
       [0.83401113, 0.93850879, 0.65489439, 0.56968483, 0.83637021,
        0.0593352 , 0.80468949, 0.25579409, 0.83520384, 0.63148784],
       [0.75651862, 0.02906495, 0.43713587, 0.09050017, 0.99865003,
        0.1294967 , 0.54700504, 0.16572704, 0.61749   , 0.93348866],
       [0.64794983, 0.86036583, 0.5459406 , 0.62964975, 0.1557481 ,
        0.76078904, 0.05542083, 0.45285043, 0.79460447, 0.07949723],
       [0.21883819, 0.29069411, 0.82861774, 0.74866212, 0.52667036,
        0.92539858, 0.24732184, 0.98072277, 0.39762179, 0.86201217],
       [0.16806775, 0.42584018, 0.00308402, 0.83713244, 0.06811465,
        0.53895836, 0.12390231, 0.00445061, 0.47946438, 0.14733311],
       [0.9273137 , 0.72902687, 0.1950168